In [4]:
import os
from google.cloud import bigquery
import pandas as pd

In [8]:
credentials = os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="ETL-setup.json"

In [26]:
#load data from local machine to bigquery - staging area
def load_csv_to_bigquery(csv_path, project_id, table_name):
    dataset_name = 'staging'
    # Create a BigQuery client using your service account key file
    #credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(project=project_id)

    # Read the CSV file into a Pandas dataframe
    df = pd.read_csv(csv_path, encoding='ISO-8859-1')

    # Create the BigQuery dataset if it doesn't exist
    dataset_ref = client.dataset(dataset_name)
    try:
        client.get_dataset(dataset_ref)
        print("Dataset {} already exists".format(dataset_name))
    except:
        print("Creating dataset {}".format(dataset_name))
        dataset = bigquery.Dataset(dataset_ref)
        client.create_dataset(dataset)

    # Set the destination table for the data
    table_ref = dataset_ref.table(table_name)

    # Define the schema of the table
    schema = []
    for column in df.columns:
        schema.append(bigquery.SchemaField(column, 'STRING'))

    # Create the table in BigQuery
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)

    # Load the data into the table
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.skip_leading_rows = 1
    job_config.autodetect = False # Set to True to automatically detect schema, False to use schema defined above
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()

    print("Data uploaded to BigQuery successfully.")

In [95]:
def clean_bigquery_table(project_id, table_id, remove_nulls=False, remove_duplicates=False, date_columns=None, columns_to_check=None):
    """
    Clean a BigQuery table by removing null values and/or duplicates.

    Args:
        project_id (str): The Google Cloud Project ID.
        table_id (str): The BigQuery table ID.
        remove_nulls (bool, optional): Whether to remove rows with null values. Defaults to False.
        columns_to_check (list, optional): List of columns to check for null values or duplicates. Defaults to None (all columns).
        remove_duplicates (bool, optional): Whether to remove duplicate rows. Defaults to False.
        date_columns (list, optional): List of columns to convert to date format. Defaults to None.

    Returns:
        None
    """
    client = bigquery.Client()
    table_ref = client.get_table(table_id)
    table = client.get_table(table_ref)

    if columns_to_check is None:
        columns_to_check = [field.name for field in table.schema]

    sql_base = f"SELECT * FROM `{table_id}`"
    sql_conditions = []

    if remove_nulls:
        not_null_conditions = [f"{column} IS NOT NULL" for column in columns_to_check]
        sql_conditions.append(" AND ".join(not_null_conditions))

    if remove_duplicates:
        deduplicate_clause = "SELECT DISTINCT"
    else:
        deduplicate_clause = "SELECT"

    if sql_conditions:
        sql_condition = "WHERE " + " AND ".join(sql_conditions)
    else:
        sql_condition = ""

    # Handle date column transformation, and make all columns lower case
    select_columns = []
    for column in table.schema:
        if column.name in date_columns:
            select_columns.append(f"PARSE_DATE('%d-%m-%Y', REGEXP_REPLACE({column.name}, r'/', '-')) AS {column.name.lower()}")
        else:
            select_columns.append(column.name.lower())


        sql = f"{deduplicate_clause} {', '.join(select_columns)} FROM ({sql_base}) AS subquery {sql_condition}"

    # Execute the query and save the results to a new table
    new_table_id = f"{project_id}.{table_ref.dataset_id}.{table_ref.table_id}_cleaned"
    new_table_ref = client.dataset(table_ref.dataset_id).table(f"{table_ref.table_id}_cleaned")

    job_config = bigquery.QueryJobConfig(destination=new_table_ref)
    query_job = client.query(sql, job_config=job_config)
    query_job.result()

    print(f"Cleaned table saved as {new_table_id}.")


In [86]:
load_csv_to_bigquery("C:\\Users\\UX501VW\\Desktop\\BigQuery-Data-Modeling\\Dastaset\\superstore_dataset2011-2015.csv", "snappy-nomad-382716", "superstore")

Dataset staging already exists


Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/snappy-nomad-382716/datasets/staging/tables?prettyPrint=false: Already Exists: Table snappy-nomad-382716:staging.superstore

In [96]:
clean_bigquery_table("snappy-nomad-382716", "snappy-nomad-382716.staging.superstore", remove_nulls=True, remove_duplicates=True, date_columns=["Order_Date", "Ship_Date"], columns_to_check=["Customer_ID", "Order_Date", "Order_ID", "Product_ID"])

Cleaned table saved as snappy-nomad-382716.staging.superstore_cleaned.


In [35]:
fact_table_name = "sales_fact"
fact_table_columns = [
    bigquery.SchemaField("order_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("product_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("customer_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("sales", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("profit", "FLOAT", mode="NULLABLE"),
]

dimension_tables = {
    "date_dim": [
        bigquery.SchemaField("date", "DATE", mode="REQUIRED"),
        bigquery.SchemaField("day", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("month", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("year", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("quarter", "INTEGER", mode="NULLABLE"),
    ],
    "customer_dim": [
        bigquery.SchemaField("customer_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("customer_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("segment", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("country", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("city", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("state", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("postal_code", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("region", "STRING", mode="NULLABLE"),
    ],
    "product_dim": [
        bigquery.SchemaField("product_id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("category", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("sub_category", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("product_name", "STRING", mode="NULLABLE"),
    ],
}

fact_dimension_key_map = {
    "order_id": {
        "date_dim_date": "date_dim",
        "customer_dim_customer_id": "customer_dim",
        "product_dim_product_id": "product_dim",
    }
}

In [36]:
from google.cloud import bigquery

def create_star_schema_from_columns(
    project_id,
    staging_dataset_id,
    source_table_name,
    fact_table_name,
    fact_table_columns,
    dimension_table_columns_map
):
    # Create a client
    client = bigquery.Client(project=project_id)
    staging_dataset_ref = client.dataset(staging_dataset_id)

    # Create the warehouse dataset
    warehouse_dataset_id = f"{staging_dataset_id}_warehouse"
    warehouse_dataset_ref = client.dataset(warehouse_dataset_id)
    try:
        client.get_dataset(warehouse_dataset_ref)
        print(f"Warehouse dataset {warehouse_dataset_id} already exists.")
    except NotFound:
        warehouse_dataset = bigquery.Dataset(warehouse_dataset_ref)
        client.create_dataset(warehouse_dataset)
        print(f"Warehouse dataset {warehouse_dataset_id} created.")
    
    # Get the source table
    source_table_ref = staging_dataset_ref.table(source_table_name)
    source_table = client.get_table(source_table_ref)

    # Create fact table
    fact_table_ref = warehouse_dataset_ref.table(fact_table_name)
    fact_table = bigquery.Table(fact_table_ref, schema=fact_table_columns)
    client.create_table(fact_table)
    print(f"Fact table {fact_table_name} created.")

    # Create dimension tables
    for dimension_table_name, dimension_columns in dimension_table_columns_map.items():
        dimension_table_ref = warehouse_dataset_ref.table(dimension_table_name)
        dimension_table = bigquery.Table(dimension_table_ref, schema=dimension_columns)
        client.create_table(dimension_table)
        print(f"Dimension table {dimension_table_name} created.")


In [38]:
create_star_schema_from_columns(
    project_id="snappy-nomad-382716",
    staging_dataset_id="snappy-nomad-382716.staging",
    source_table_name="superstore_cleaned",
    fact_table_name=fact_table_name,
    fact_table_columns=fact_table_columns,
    dimension_table_columns_map=dimension_tables,
)

NameError: name 'NotFound' is not defined

In [61]:

# def create_star_schema(
#     project_id,
#     staging_table_id,
#     facts_table_name,
#     facts_columns,
#     dimensions,
#     warehouse_dataset_name="warehouse"
# ):
#     client = bigquery.Client(project=project_id)

#     # # Create the warehouse dataset if it doesn't exist
#     # warehouse_dataset_id = f"{project_id}.{warehouse_dataset_name}"
#     # try:
#     #     client.get_dataset(warehouse_dataset_id)
#     # except NotFound:
#     #     warehouse_dataset = bigquery.Dataset(warehouse_dataset_id)
#     #     client.create_dataset(warehouse_dataset)
#     #     print(f"Created dataset '{warehouse_dataset_name}'.")
        
        
#     dataset_ref = client.dataset(warehouse_dataset_name)
#     try:
#         client.get_dataset(dataset_ref)
#         print("Dataset {} already exists".format(warehouse_dataset_name))
#     except:
#         print("Creating dataset {}".format(warehouse_dataset_name))
#         dataset = bigquery.Dataset(dataset_ref)
#         client.create_dataset(dataset)

#     # Create the facts table
#     facts_table_id = f"{warehouse_dataset_name}.{facts_table_name}"
#     facts_schema = [bigquery.SchemaField(col_name, col_type) for col_name, col_type in facts_columns]
#     facts_table = bigquery.Table(facts_table_id, schema=facts_schema)
#     client.create_table(facts_table)
#     print(f"Created table '{facts_table_name}'.")

#     # Create the dimensions tables
#     for dimension in dimensions:
#         dim_table_name, dim_columns = dimension
#         dim_table_id = f"{warehouse_dataset_name}.{dim_table_name}"
#         dim_schema = [bigquery.SchemaField(col_name, col_type) for col_name, col_type in dim_columns]
#         dim_table = bigquery.Table(dim_table_id, schema=dim_schema)
#         client.create_table(dim_table)
#         print(f"Created table '{dim_table_name}'.")
        
        
#from google.cloud import bigquery
# from google.api_core.exceptions import NotFound

# def create_star_schema(
#     project_id,
#     staging_table_id,
#     facts_table_name,
#     facts_columns,
#     dimensions,
#     warehouse_dataset_name="warehouse"
# ):
#     client = bigquery.Client(project=project_id)

#     # Create the warehouse dataset if it doesn't exist
#         dataset_ref = client.dataset(warehouse_dataset_name)
#     try:
#         client.get_dataset(dataset_ref)
#         print("Dataset {} already exists".format(warehouse_dataset_name))
#     except:
#         print("Creating dataset {}".format(warehouse_dataset_name))
#         dataset = bigquery.Dataset(dataset_ref)
#         client.create_dataset(dataset)

#     # Create the facts table
#     facts_table_id = f"{project_id}.{warehouse_dataset_name}.{facts_table_name}"
#     facts_schema = [bigquery.SchemaField(col_name, col_type) for col_name, col_type in facts_columns]
#     facts_table = bigquery.Table(facts_table_id, schema=facts_schema)
#     client.create_table(facts_table)
#     print(f"Created table '{facts_table_name}'.")

#     # Create the dimensions tables
#     for dimension in dimensions:
#         dim_table_name, dim_columns = dimension
#         dim_table_id = f"{project_id}.{warehouse_dataset_name}.{dim_table_name}"
#         dim_schema = [bigquery.SchemaField(col_name, col_type) for col_name, col_type in dim_columns]
#         dim_table = bigquery.Table(dim_table_id, schema=dim_schema)
#         client.create_table(dim_table)
#         print(f"Created table '{dim_table_name}'.")

#from google.cloud import bigquery
# from google.api_core.exceptions import NotFound

# def create_star_schema(
#     project_id,
#     staging_table_id,
#     facts_table_name,
#     facts_columns,
#     dimensions,
#     warehouse_dataset_name="warehouse"
# ):
#     client = bigquery.Client(project=project_id)

#     # Create the warehouse dataset if it doesn't exist
#     warehouse_dataset_id = f"{project_id}.{warehouse_dataset_name}"
#     try:
#         client.get_dataset(warehouse_dataset_id)
#     except NotFound:
#         warehouse_dataset = bigquery.Dataset(warehouse_dataset_id)
#         client.create_dataset(warehouse_dataset)
#         print(f"Created dataset '{warehouse_dataset_name}'.")

#     # Create the facts table by selecting the required columns from the staging table
#     facts_table_id = f"{project_id}.{warehouse_dataset_name}.{facts_table_name}"
#     facts_select_columns = ', '.join([f"staging.{col_name}" for col_name, _ in facts_columns])

#     facts_query = f"""
#         CREATE TABLE `{facts_table_id}`
#         AS SELECT {facts_select_columns}
#         FROM `{staging_table_id}` staging
#     """
#     client.query(facts_query).result()
#     print(f"Created table '{facts_table_name}'.")

#     # Create the dimensions tables
#     for dimension in dimensions:
#         dim_table_name, dim_columns = dimension
#         dim_table_id = f"{project_id}.{warehouse_dataset_name}.{dim_table_name}"
#         dim_select_columns = ', '.join([f"staging.{col_name}" for col_name, _ in dim_columns])

#         dim_query = f"""
#             CREATE TABLE `{dim_table_id}`
#             AS SELECT DISTINCT {dim_select_columns}
#             FROM `{staging_table_id}` staging
#         """
#         client.query(dim_query).result()
#         print(f"Created table '{dim_table_name}'.")

from google.cloud import bigquery
from google.api_core.exceptions import NotFound

def create_star_schema(
    project_id,
    staging_table_id,
    facts_table_name,
    facts_columns,
    dimensions,
    warehouse_dataset_name="warehouse"
):
    client = bigquery.Client(project=project_id)

    # Create the warehouse dataset if it doesn't exist
    warehouse_dataset_id = f"{project_id}.{warehouse_dataset_name}"
    try:
        client.get_dataset(warehouse_dataset_id)
    except NotFound:
        warehouse_dataset = bigquery.Dataset(warehouse_dataset_id)
        client.create_dataset(warehouse_dataset)
        print(f"Created dataset '{warehouse_dataset_name}'.")

    # Create the facts table by selecting the required columns from the staging table
    facts_table_id = f"{project_id}.{warehouse_dataset_name}.{facts_table_name}"
    facts_select_columns = ', '.join([f"staging.{col_name}" for col_name, _ in facts_columns])

    facts_query = f"""
        CREATE TABLE `{facts_table_id}`
        AS SELECT {facts_select_columns}
        FROM `{staging_table_id}` staging
    """
    client.query(facts_query).result()
    print(f"Created table '{facts_table_name}'.")

    # Create the dimensions tables
    for dimension in dimensions:
        dim_table_name, dim_columns = dimension
        dim_table_id = f"{project_id}.{warehouse_dataset_name}.{dim_table_name}"
        dim_select_columns = ', '.join([f"staging.{col_name}" for col_name, _ in dim_columns])

        dim_query = f"""
            CREATE TABLE `{dim_table_id}`
            AS SELECT DISTINCT {dim_select_columns}
            FROM `{staging_table_id}` staging
        """
        client.query(dim_query).result()
        print(f"Created table '{dim_table_name}'.")




In [62]:
def create_superstore_star_schema(project_id, staging_table_id):
    facts_table_name = "order_facts"
    facts_columns = [
        ("order_id", "STRING"),
        ("order_date", "DATE"),
        ("ship_date", "DATE"),
        ("sales", "FLOAT64"),
        ("quantity", "INT64"),
        ("discount", "FLOAT64"),
        ("profit", "FLOAT64"),
        ("customer_id", "STRING"),
        ("product_id", "STRING"),
    ]

    
    dimensions = [
        (
            "customers",
            [
                ("customer_id", "STRING"),
                ("customer_name", "STRING"),
                ("segment", "STRING"),
                #("region", "STRING")
                ("country", "STRING"),
                ("state", "STRING"),
                ("city", "STRING"),
                ("postal_code", "STRING")
                
            ],
        ),
        (
            "products",
            [
                ("product_id", "STRING"),
                ("product_name", "STRING"),
                ("category", "STRING"),
                ("sub_category", "STRING")
            ],
        ),
        (
            "date_dim",
            [
                ("order_date", "DATE"),
                ("ship_date", "DATE")
                # ("day", "INTEGER"),
                # ("month", "INTEGER"),
                # ("year", "INTEGER"),
            ],
        ),
        
    ]

    create_star_schema(
        project_id=project_id,
        staging_table_id=staging_table_id,
        facts_table_name=facts_table_name,
        facts_columns=facts_columns,
        dimensions=dimensions,
    )


In [63]:
create_superstore_star_schema("snappy-nomad-382716", "snappy-nomad-382716.staging.superstore_cleaned")

Created table 'order_facts'.
Created table 'customers'.
Created table 'products'.
Created table 'date_dim'.
